In [1]:
from dataset_slide_turn_change import *
import torch
import torch.nn as nn

import numpy as np

from scipy.stats import uniform, randint

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error

Loading...
Finished Loading...


In [2]:
person_order = {'F1_Interaction_1': {'P2': 1, 'P1': 1, 'P3': 2},
 'F1_Interaction_2': {'P2': 1, 'P1': 1, 'P3': 2},
 'F2_Interaction_1': {'P4': 1, 'P5': 3},
 'F2_Interaction_2': {'P4': 1},
 'F3_Interaction_1': {'P8': 3, 'P6': 1, 'P7': 1},
 'F3_Interaction_2': {'P6': 1, 'P7': 1},
 'F4_Interaction_1': {'P14': 2,
  'P12': 1,
  'P11': 1,
  'P10': 1,
  'P9': 1,
  'P13': 3},
 'F4_Interaction_2': {'P12': 1,
  'P11': 1,
  'P10': 1,
  'P9': 1,
  'P13': 3},
 'F5_Interaction_1': {'P16': 2, 'P15': 1},
 'F5_Interaction_2': {'P16': 2, 'P15': 1},
 'F6_Interaction_1': {'P19': 3, 'P18': 1, 'P17': 1},
 'F6_Interaction_2': {'P19': 3, 'P18': 1, 'P17': 1},
 'F7_Interaction_1': {'P22': 3,
  'P20': 1,
  'P21': 1,
  'P23': 2},
 'F8_Interaction_1': {'P24': 1, 'P25': 3},
 'F8_Interaction_2': {'P24': 1, 'P25': 3},
 'F8_Interaction_3': {'P24': 1, 'P25': 3},
 'F10_Interaction_1': {'P27': 1, 'P28': 1},
 'F11_Interaction_1': {'P29': 1, 'P30': 2},
 'F11_Interaction_2': {'P29': 1, 'P30': 2},
 'F13_Interaction_1': {'P32': 1, 'P33': 2},
 'F17_Interaction_1': {'P37': 1, 'P38': 2},
 'F17_Interaction_2': {'P37': 1, 'P38': 2}}


group_nums = {1: ['F2_Interaction_2'],
 2: ['F2_Interaction_1',
  'F3_Interaction_2',
  'F5_Interaction_1',
  'F5_Interaction_2',
  'F8_Interaction_1',
  'F8_Interaction_2',
  'F8_Interaction_3',
  'F10_Interaction_1',
  'F11_Interaction_1',
  'F11_Interaction_2',
  'F13_Interaction_1',
  'F17_Interaction_1',
  'F17_Interaction_2'],
 3: ['F1_Interaction_1',
  'F1_Interaction_2',
  'F3_Interaction_1',
  'F6_Interaction_1',
  'F6_Interaction_2'],
 4: ['F7_Interaction_1'],
 5: ['F4_Interaction_2'],
 6: ['F4_Interaction_1']}

group_all_dataset = []
group_ids = group_nums[3]
for group_id in group_ids:
    group_specific_dataset = SpeedDatingDS(group_id = group_id, social_rel = person_order[group_id])
    group_all_dataset.append(group_specific_dataset)

SD = torch.utils.data.ConcatDataset(group_all_dataset)

########################################################################
#Dataloader
########################################################################
train_len = len(SD) - len(SD)//5
test_len = len(SD)//5

train, test = torch.utils.data.random_split(SD, (train_len, test_len), generator=torch.Generator().manual_seed(0))

batch_size = 32
trainloader = DataLoader(train, batch_size = train_len, shuffle = True, num_workers = 8)
testloader = DataLoader(test, batch_size = test_len, shuffle = True, num_workers = 8)

In [7]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for idx, batch in enumerate(trainloader):

    x_train, vb_output = batch['context'], batch['vb_output']

    labels = vb_output.sum(2).to(device).flatten(start_dim =1)
    index_labels = torch.zeros(x_train.shape[0]).long().to(device)
    index_labels[labels.nonzero()[:,0]] = labels.nonzero()[:,1] + 1 
    y_train = index_labels

for idx, batch in enumerate(testloader):
    x_test, vb_output = batch['context'], batch['vb_output']

    labels = vb_output.sum(2).to(device).flatten(start_dim =1)
    index_labels = torch.zeros(x_test.shape[0]).long().to(device)
    index_labels[labels.nonzero()[:,0]] = labels.nonzero()[:,1] + 1 
    y_test = index_labels




In [8]:
# x = x_train
# y = y_train

# x_test = x_test
# y_test = y_test.cpu().numpy()

In [44]:
x_test[...,desired_ind].flatten(start_dim =1).cpu().numpy().shape

(665, 3)

In [24]:
behavior_types = {
    'Acknowledgement' : [*range(1,7)],
    'Body' : [*range(7,14)],
    'Head' : [*range(14, 20)],
    'Hand' : [*range(20,22)],
    'Eye' : [*range(22,25)],
    'Face' : [*range(25,30)],
    'Positive_Verbal' : [*range(30,35)],
    'Negative_Verbal' : [*range(35,39)]} 


#mouth gape

input_feats = 38
desired_ind = [28]
x_abl = x_train[...,desired_ind].flatten(start_dim =1).cpu().numpy()
y_abl = y_train.cpu().numpy()
x_test_abl = x_test[...,desired_ind].flatten(start_dim =1).cpu().numpy()
y_test_abl = y_test.cpu().numpy()

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import sklearn

print(__doc__)


# Set the parameters by cross-validation
tuned_parameters = [
                    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel':['sigmoid'], 'gamma':["auto"]},
                    {'kernel':['poly'], 'gamma':[1e-1 , 1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
                   ]

scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(x_abl, y_abl)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test_abl, clf.predict(x_test_abl)
    print("f1: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='macro')))
    print("f1_weighted: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='weighted')))
    print("acc: {}".format((y_pred == y_true).mean()))
    print(confusion_matrix(y_test_abl, y_pred))

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.189 (+/-0.010) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.189 (+/-0.010) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.189 (+/-0.010) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.189 (+/-0.010) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.189 (+/-0.010) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.189 (+/-0.010) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.210 (+/-0.079) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.189 (+/-0.010) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.210 (+/-0.079) for {'C': 1, 'kernel': 'linear'}
0.210 (+/-0.079) for {'C': 10, 'kernel': 'linear'}
0.210 (+/-0.079) for {'C': 100, 'kernel': 'linear'}
0.210 (+/-0.079) for {'C': 1000, 'kernel': 'linear'}
0.189 (+/-0.010) for {'gamma': 'auto', 'kernel': 

In [25]:
behavior_types = {
    'Acknowledgement' : [*range(1,7)],
    'Body' : [*range(7,14)],
    'Head' : [*range(14, 20)],
    'Hand' : [*range(20,22)],
    'Eye' : [*range(22,25)],
    'Face' : [*range(25,30)],
    'Positive_Verbal' : [*range(30,35)],
    'Negative_Verbal' : [*range(35,39)]} 


#head movement

input_feats = 38
desired_ind = [range(13,19)]
x_abl = x_train[...,desired_ind].flatten(start_dim =1).cpu().numpy()
y_abl = y_train.cpu().numpy()
x_test_abl = x_test[...,desired_ind].flatten(start_dim =1).cpu().numpy()
y_test_abl = y_test.cpu().numpy()

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import sklearn

print(__doc__)


# Set the parameters by cross-validation
tuned_parameters = [
#                     {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
#                     {'kernel':['sigmoid'], 'gamma':["auto"]},
                     {'kernel':['poly'], 'gamma':[1e-1 , 1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
                   ]

scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(x_abl, y_abl)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test_abl, clf.predict(x_test_abl)
    print("f1: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='macro')))
    print("f1_weighted: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='weighted')))
    print("acc: {}".format((y_pred == y_true).mean()))
    print(confusion_matrix(y_test_abl, y_pred))

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.1, 'kernel': 'poly'}

Grid scores on development set:

0.184 (+/-0.004) for {'C': 1, 'gamma': 0.1, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 1, 'gamma': 0.01, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 1, 'gamma': 0.0001, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 10, 'gamma': 0.01, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 10, 'gamma': 0.001, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 10, 'gamma': 0.0001, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 100, 'gamma': 0.1, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 100, 'gamma': 0.01, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 100, 'gamma': 0.001, 'kernel': 'poly'}
0.184 (+/-0.004) for {'C': 100, 'gamma': 0.0001, 'kernel': 'poly'}
0.

In [26]:
behavior_types = {
    'Acknowledgement' : [*range(1,7)],
    'Body' : [*range(7,14)],
    'Head' : [*range(14, 20)],
    'Hand' : [*range(20,22)],
    'Eye' : [*range(22,25)],
    'Face' : [*range(25,30)],
    'Positive_Verbal' : [*range(30,35)],
    'Negative_Verbal' : [*range(35,39)]} 


#gaze, prosody, head

input_feats = 38
desired_ind = [*range(13,19)] + [*range(0,6)] + [*range(21,24)] + [*range(29,38)]
x_abl = x_train[...,desired_ind].flatten(start_dim =1).cpu().numpy()
y_abl = y_train.cpu().numpy()
x_test_abl = x_test[...,desired_ind].flatten(start_dim =1).cpu().numpy()
y_test_abl = y_test.cpu().numpy()

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import sklearn

print(__doc__)


# Set the parameters by cross-validation
tuned_parameters = [
                    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel':['sigmoid'], 'gamma':["auto"]},
                     {'kernel':['poly'], 'gamma':[1e-1 , 1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
                   ]

scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(x_abl, y_abl)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test_abl, clf.predict(x_test_abl)
    print("f1: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='macro')))
    print("f1_weighted: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='weighted')))
    print("acc: {}".format((y_pred == y_true).mean()))
    print(confusion_matrix(y_test_abl, y_pred))

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'C': 100, 'kernel': 'linear'}

Grid scores on development set:

0.241 (+/-0.137) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.241 (+/-0.137) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.241 (+/-0.137) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.241 (+/-0.137) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.437 (+/-0.161) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.241 (+/-0.137) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.498 (+/-0.166) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.437 (+/-0.161) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.532 (+/-0.139) for {'C': 1, 'kernel': 'linear'}
0.534 (+/-0.095) for {'C': 10, 'kernel': 'linear'}
0.540 (+/-0.101) for {'C': 100, 'kernel': 'linear'}
0.540 (+/-0.101) for {'C': 1000, 'kernel': 'linear'}
0.241 (+/-0.137) for {'gamma': 'auto', 'kernel': 'sigmoid'}
0.2

In [27]:
behavior_types = {
    'Acknowledgement' : [*range(1,7)],
    'Body' : [*range(7,14)],
    'Head' : [*range(14, 20)],
    'Hand' : [*range(20,22)],
    'Eye' : [*range(22,25)],
    'Face' : [*range(25,30)],
    'Positive_Verbal' : [*range(30,35)],
    'Negative_Verbal' : [*range(35,39)]} 


# all

x_abl = x_train.flatten(start_dim =1).cpu().numpy()
y_abl = y_train.cpu().numpy()
x_test_abl = x_test.flatten(start_dim =1).cpu().numpy()
y_test_abl = y_test.cpu().numpy()

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import sklearn

print(__doc__)


# Set the parameters by cross-validation
tuned_parameters = [
                    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel':['sigmoid'], 'gamma':["auto"]},
                     {'kernel':['poly'], 'gamma':[1e-1 , 1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
                   ]

scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(x_abl, y_abl)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test_abl, clf.predict(x_test_abl)
    print("f1: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='macro')))
    print("f1_weighted: {}".format(sklearn.metrics.f1_score(y_pred, y_true, average='weighted')))
    print("acc: {}".format((y_pred == y_true).mean()))
    print(confusion_matrix(y_test_abl, y_pred))

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'C': 10, 'kernel': 'linear'}

Grid scores on development set:

0.250 (+/-0.164) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.250 (+/-0.164) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.250 (+/-0.164) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.250 (+/-0.164) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.599 (+/-0.242) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.250 (+/-0.164) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.626 (+/-0.050) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.599 (+/-0.242) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.608 (+/-0.105) for {'C': 1, 'kernel': 'linear'}
0.629 (+/-0.090) for {'C': 10, 'kernel': 'linear'}
0.619 (+/-0.096) for {'C': 100, 'kernel': 'linear'}
0.619 (+/-0.096) for {'C': 1000, 'kernel': 'linear'}
0.250 (+/-0.164) for {'gamma': 'auto', 'kernel': 'sigmoid'}
0.22